In [16]:
import torch
import torch.nn as nn
import torch.optim as optim
from rumour_milled.load import load_headlines
from transformers import AutoTokenizer, AutoModel

In [2]:
# GeForce Game Ready Driver 577.0
# GTX 1660
# Turing 7.5
# CUDA SDK 10.0-10.2
torch.device("cuda" if torch.cuda.is_available() else "cpu")#

device(type='cuda')

In [18]:
tokeniser = AutoTokenizer.from_pretrained("bert-base-uncased")
vectoriser = AutoModel.from_pretrained("bert-base-uncased")
headlines, labels = load_headlines()

In [ ]:
tokens = tokeniser(
    headlines[:100], 
    padding=True,
    truncation=True,
    return_tensors="pt"
)

with torch.no_grad():
    outputs = vectoriser(**tokens)

In [81]:
def tokenise_and_vectorise(headline):
    tokens = tokeniser(
        headline, 
        padding=True,
        truncation=True,
        return_tensors="pt"
    )
    with torch.no_grad():
        outputs = vectoriser(**tokens)
    return outputs.last_hidden_state[:, 0, :]

In [74]:
X = outputs.last_hidden_state[:, 0, :]
y = torch.tensor(labels[:100], dtype=torch.float).unsqueeze(1)

In [75]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X, y)
data_loader = DataLoader(dataset, batch_size=10, shuffle=True)

In [76]:
# Define a 2-hidden-layer network
class HeadlineClassifier(nn.Module):
    def __init__(self):
        super(HeadlineClassifier, self).__init__()
        self.fc1 = nn.Linear(768, 256)  # Input layer -> 1st hidden layer
        self.fc2 = nn.Linear(256, 8)  # 1st hidden -> 2nd hidden
        self.fc3 = nn.Linear(8, 1)   # 2nd hidden -> Output layer

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

In [100]:
model = HeadlineClassifier()
criterion = nn.BCEWithLogitsLoss()  # Binary Cross Entropy for [0,1] output
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [101]:
# Training loop
for epoch in range(1000):
    total_loss = 0
    for batch_X, batch_y in data_loader:
        optimizer.zero_grad()
        logits = model(batch_X)
        loss = criterion(logits, batch_y)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1}, Loss: {total_loss:.4f}")

Epoch 1, Loss: 7.1471
Epoch 2, Loss: 6.5709
Epoch 3, Loss: 6.0164
Epoch 4, Loss: 5.6329
Epoch 5, Loss: 5.5163
Epoch 6, Loss: 5.4079
Epoch 7, Loss: 5.3657
Epoch 8, Loss: 5.2754
Epoch 9, Loss: 5.2317
Epoch 10, Loss: 5.2061
Epoch 11, Loss: 5.1884
Epoch 12, Loss: 5.1759
Epoch 13, Loss: 5.1708
Epoch 14, Loss: 5.1655
Epoch 15, Loss: 5.1618
Epoch 16, Loss: 5.1600
Epoch 17, Loss: 5.1572
Epoch 18, Loss: 5.1563
Epoch 19, Loss: 5.1548
Epoch 20, Loss: 5.1540
Epoch 21, Loss: 5.1532
Epoch 22, Loss: 5.1524
Epoch 23, Loss: 5.1518
Epoch 24, Loss: 5.1514
Epoch 25, Loss: 5.1508
Epoch 26, Loss: 5.1504
Epoch 27, Loss: 5.1501
Epoch 28, Loss: 5.1498
Epoch 29, Loss: 5.1496
Epoch 30, Loss: 5.1492
Epoch 31, Loss: 5.1490
Epoch 32, Loss: 5.1489
Epoch 33, Loss: 5.1487
Epoch 34, Loss: 5.1485
Epoch 35, Loss: 5.1484
Epoch 36, Loss: 5.1482
Epoch 37, Loss: 5.1481
Epoch 38, Loss: 5.1480
Epoch 39, Loss: 5.1479
Epoch 40, Loss: 5.1478
Epoch 41, Loss: 5.1477
Epoch 42, Loss: 5.1476
Epoch 43, Loss: 5.1475
Epoch 44, Loss: 5.14

In [102]:
with torch.no_grad():
    preds = model(tokenise_and_vectorise("Earth shattering headline"))
    print("Predictions:", preds.round())

Predictions: tensor([[0.]])
